In [91]:
import pandas as pd
from sklearn import datasets
import numpy as np
import random

iris = datasets.load_iris()
irisDF = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])

irisDF['target_name'] = ''

for i in irisDF.index:
    irisDF.target_name[i] = iris['target_names'][int(irisDF.target[i])]

irisDF = irisDF.sample(frac=1,random_state=1).reset_index(drop=True)

print(irisDF)

trainIris = irisDF[:110]

testIris = irisDF[110:].reset_index(drop=True)
testIris['hyp'] = np.nan


     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.8               4.0                1.2               0.2   
1                  5.1               2.5                3.0               1.1   
2                  6.6               3.0                4.4               1.4   
3                  5.4               3.9                1.3               0.4   
4                  7.9               3.8                6.4               2.0   
..                 ...               ...                ...               ...   
145                6.3               2.8                5.1               1.5   
146                6.4               3.1                5.5               1.8   
147                6.3               2.5                4.9               1.5   
148                6.7               3.1                5.6               2.4   
149                4.9               3.6                1.4               0.1   

     target target_name  
0

C:\Users\romai\AppData\Local\Temp\ipykernel_3388\2912078258.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  irisDF.target_name[i] = iris['target_names'][int(irisDF.target[i])]


In [179]:
def stats(test):
    pasBon = 0
    
    bonParClasse = {}
    for c in test.target.unique():
            bonParClasse[c] = 0

    for i in test.index:

        if (test.target[i] == test.hyp[i]):
            bonParClasse[test.hyp[i]] += 1
        else:
            pasBon += 1

    #Calcul précision
    precision = 0
    for c in bonParClasse.keys():
        #precision += bonParClasse[c] / test.hyp.value_counts


    precision /= len(test)

    return {'Erreur': pasBon/(len(test)), 'Precision': precision}

stats(testIris)

IndentationError: expected an indented block after 'for' statement on line 17 (3696638946.py, line 21)

In [177]:
def trainZeroR(train):
    return np.argmax(np.bincount(train.target))
    
def testIrisZeroR(test, model):
    for i in test.index:
        test.hyp[i] = model

In [178]:
model = trainZeroR(trainIris)

testIrisZeroR(testIris, model)

print(testIris)


stats(testIris)

    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                 5.6               2.8                4.9               2.0   
1                 6.8               3.2                5.9               2.3   
2                 5.0               3.3                1.4               0.2   
3                 5.1               3.7                1.5               0.4   
4                 5.9               3.2                4.8               1.8   
5                 4.6               3.1                1.5               0.2   
6                 5.8               2.7                5.1               1.9   
7                 4.8               3.1                1.6               0.2   
8                 6.5               3.0                5.2               2.0   
9                 4.9               2.5                4.5               1.7   
10                4.6               3.2                1.4               0.2   
11                6.4               3.2 

C:\Users\romai\AppData\Local\Temp\ipykernel_3388\1131553383.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.hyp[i] = model


{'Erreur': 0.7, 'Precision': 0.0}

In [94]:
def discretize(data, means, featuresToDiscretize):
    
    dataDiscret = data.copy()
    
    for col in featuresToDiscretize:
        for i in data.index:
            
            if (data[col][i] >= means[col]):
                dataDiscret[col][i] = "long"
            else:
                dataDiscret[col][i] = "court"
                
    return dataDiscret


trainIrisDis = discretize(trainIris, trainIris.mean(), iris['feature_names'])
testIrisDis = discretize(testIris, trainIris.mean(), iris['feature_names'])


print(testIrisDis)

   sepal length (cm) sepal width (cm) petal length (cm) petal width (cm)  \
0              court            court              long             long   
1               long             long              long             long   
2              court             long             court            court   
3              court             long             court            court   
4              court             long              long             long   
5              court             long             court            court   
6              court            court              long             long   
7              court             long             court            court   
8               long            court              long             long   
9              court            court              long             long   
10             court             long             court            court   
11              long             long              long             long   
12          

C:\Users\romai\AppData\Local\Temp\ipykernel_3388\1950391893.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  trainIrisDis = discretize(trainIris, trainIris.mean(), iris['feature_names'])
C:\Users\romai\AppData\Local\Temp\ipykernel_3388\1950391893.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataDiscret[col][i] = "court"
C:\Users\romai\AppData\Local\Temp\ipykernel_3388\1950391893.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [96]:
def trainOneR(train, featuresColumns):
    
    bestPrecision = -1

    for col in featuresColumns:
        res = pd.DataFrame(data = np.zeros((len(train.target.unique()), len(train[col].unique()))), columns = train[col].unique())

        for i in train.index:
            res[train[col][i]][int(train.target[i])] += 1
        
        precision = res.max().sum() / res.sum().sum()

        #Update du model avec nouvelle colone
        if (precision > bestPrecision):
            bestPrecision = precision
            model = {'feature_name': col}
            
            for value in res:
                print()
                model[value] = res.idxmax()[value]

    return model



In [154]:
def testOneR(test, model):
    for i in test.index:
        test.hyp[i] = model[ test[ model['feature_name'] ][i] ]

model = trainOneR(trainIrisDis, iris["feature_names"])
print(model)
    
testOneR(testIrisDis, model)
testIrisDis.hyp = np.nan

testOneR(testIrisDis, model)
print(testIrisDis)

stats(testIrisDis)






{'feature_name': 'petal length (cm)', 'court': 0, 'long': 2}
   sepal length (cm) sepal width (cm) petal length (cm) petal width (cm)  \
0              court            court              long             long   
1               long             long              long             long   
2              court             long             court            court   
3              court             long             court            court   
4              court             long              long             long   
5              court             long             court            court   
6              court            court              long             long   
7              court             long             court            court   
8               long            court              long             long   
9              court            court              long             long   
10             court             long             court            court   
11              long   

C:\Users\romai\AppData\Local\Temp\ipykernel_3388\48921010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.hyp[i] = model[ test[ model['feature_name'] ][i] ]


{'precision': 0.65, 'erreur': 0.35}

In [135]:
def trainNaiveBayes(train, featuresColumns):
    
    featuresRes = {}
    bayesProbs = {}
    classesProbs = {}

    for c in train.target.unique():

        classesProbs[c] = pd.value_counts(train.target)[c] / train.target.count()

    for col in featuresColumns:
        res = pd.DataFrame(data = np.ones((len(train.target.unique()), len(train[col].unique()))), columns = train[col].unique())
        
        for i in train.index:
            res[train[col][i]][int(train.target[i])] += 1
        
        featuresRes[col] = res


    for feature in featuresRes.keys():
        resProbs = featuresRes[feature].copy()

        for ind in featuresRes[feature].index:
            for fValueName in train[col].unique():
                resProbs[fValueName][ind] = featuresRes[feature][fValueName][ind] / featuresRes[feature].sum(1)[ind]
        bayesProbs[feature] = resProbs


    model = {'featuresProbs': bayesProbs, 'classesProbs': classesProbs}

    return model

In [146]:
def testNaiveBayes(test, model):

    for i in test.index:
        
        probs = {}
        for c in test.target:

            prob=1
            for f in model['featuresProbs'].keys():
                prob *=   model['featuresProbs'][f][test[f][i]][c]
            
            prob *= model['classesProbs'][c]
            probs[c] = prob

        test.hyp[i] = max(probs, key=probs.get)

    print('ok')
    


In [155]:


model = trainNaiveBayes(trainIrisDis, iris["feature_names"])

testNaiveBayes(testIrisDis, model)

print(testIrisDis)

stats(testIrisDis)


ok
   sepal length (cm) sepal width (cm) petal length (cm) petal width (cm)  \
0              court            court              long             long   
1               long             long              long             long   
2              court             long             court            court   
3              court             long             court            court   
4              court             long              long             long   
5              court             long             court            court   
6              court            court              long             long   
7              court             long             court            court   
8               long            court              long             long   
9              court            court              long             long   
10             court             long             court            court   
11              long             long              long             long   
12       

C:\Users\romai\AppData\Local\Temp\ipykernel_3388\566371108.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.hyp[i] = max(probs, key=probs.get)


{'precision': 0.65, 'erreur': 0.35}